## Домашнее задание 26
### Козел А.Г. 22.0.2023
Решить задачу просрочки кредита используя фреймворк torch выполнить прогноз просрочки кредита заемщиком. Сравнить результаты прогноза просрочки кредита заемщиком.



In [72]:
import sys
import sklearn

import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets


import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
import tensorflow as tf
#assert tf.__version__ >= "2.0"
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
#from tensorflow.python.keras import utils

### Описание набора данных

Данные представляют собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ). Задача является бинарной классификацией. Цель - предсказать будет ли тот или иной заемщик испытывать финансовые трудности в ближайшие 2 года, т.е. будет ли просрочка по займу. Выборка разделена на тренировочную и тестовую ( 150000 в тренировочной части, 101503 в тестовой).

Данная решаемая задача будет ценна как кредиторам, так и заемщикам для оценки способности вернуть долг вовремя.

### Описание признаков

* 1 - __SeriousDlqin2yrs__ - будет ли просрочка более 90 дней в ближайшие 2 года (целевая метка) (Да/Нет)
* 2 - __RevolvingUtilizationOfUnsecuredLines__ - общий баланс по кредитным картам и кредитным линиям, за исключением задолженностей по недвижимости задолженности по взносам, деленные на сумму кредитных лимитов (проценты)
* 3 - __age__ - Возраст заемщика (в годах)
* 4 - __NumberOfTime30-59DaysPastDueNotWorse__ - количество просрочек в 30-59 дней (Целое)
* 5 - __DebtRatio__ - коэффициент задолженности, т.е. сумма ежемесячных платежей по долгам, алиментов и расходов на проживание, деленная на месячный доналоговый доход (проценты)
* 6 - __Monthly Income__ - месячный доход (число с плавающей точкой)
* 7 - __NumberOfOpenCreditLinesAndLoans__ - количество открытых кредитов и кредитных линий (Целое)
* 8 - __NumberOfTimes90DaysLate__ - количество просрочек более 90 дней (Целое)
* 9 - __NumberRealEstateLoansOrLines__ - количество ипотечных кредитов и кредитов на недвижимость, включая кредитные линии домашнего капитала (Целое)
* 10 - __NumberOfTime60-89DaysPastDueNotWorse__ - количество просрочек в 60-89 дней за последние 2 года (Целое)
* 11 - __NumberOfDependents__ - количество иждивенцев в семье (исключая самих заемщиков) (Целое)

In [6]:
df = pd.read_csv('cs-training.csv', index_col=0)


In [7]:
print(df.shape)
df.describe(include = "all").T

(150000, 11)


,count,mean,std,min,25%,50%,75%,max
SeriousDlqin2yrs,150000.0,0.066840,0.249746,0.0,0.000000,0.000000,0.000000,1.0
RevolvingUtilizationOfUnsecuredLines,150000.0,6.048438,249.755371,0.0,0.029867,0.154181,0.559046,50708.0
age,150000.0,52.295207,14.771866,0.0,41.000000,52.000000,63.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.421033,4.192781,0.0,0.000000,0.000000,0.000000,98.0
DebtRatio,150000.0,353.005076,2037.818523,0.0,0.175074,0.366508,0.868254,329664.0
MonthlyIncome,120269.0,6670.221237,14384.674215,0.0,3400.000000,5400.000000,8249.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.452760,5.145951,0.0,5.000000,8.000000,11.000000,58.0
NumberOfTimes90DaysLate,150000.0,0.265973,4.169304,0.0,0.000000,0.000000,0.000000,98.0
NumberRealEstateLoansOrLines,150000.0,1.018240,1.129771,0.0,0.000000,1.000000,2.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.240387,4.155179,0.0,0.000000,0.000000,0.000000,98.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

## Подготовка данных

In [9]:
# месячный доход пропуски заменим средним значением
df['MonthlyIncome']=df['MonthlyIncome'].fillna(df['MonthlyIncome'].median()).astype(float)

In [10]:
#NumberOfDependent (количество иждивенцев в семье) должно быть целым, а стоит число с плаввающей точкой
# преобразуем тип переменных
# пропуски заменяю на ноль, т.к.это большинство
df['NumberOfDependents']=df['NumberOfDependents'].fillna(0).astype(int)

In [11]:
# нулевой возраст быть не может, поэтому можем удалить это значение или заменить средним. удалю
df=df[(df['age']!=0)]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149999 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149999 non-null  float64
 2   age                                   149999 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149999 non-null  int64  
 4   DebtRatio                             149999 non-null  float64
 5   MonthlyIncome                         149999 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149999 non-null  int64  
 7   NumberOfTimes90DaysLate               149999 non-null  int64  
 8   NumberRealEstateLoansOrLines          149999 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149999 non-null  int64  
 10  NumberOfDependents                    149999 non-null  int32  
dtype

In [13]:
y = df['SeriousDlqin2yrs']
X = df.drop(['SeriousDlqin2yrs'], axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=42)

In [15]:
ss=StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)

# Torch

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [80]:
# проскалируем данные и разобьем на train and test
ss=StandardScaler()
X = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7, 
                                                    random_state=42, 
                                                    stratify=y)

In [81]:
class Data(Dataset):
    '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

    def __init__(self, X: np.ndarray, y: np.ndarray) -> None:
        # ужно преобразовать float64 в float32, иначе будет ошибка 
        # RuntimeError: expected scalar type Double but found Float
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
  
    def __getitem__(self, index: int) -> tuple:
        return self.X[index], self.y[index]

    def __len__(self) -> int:
        return self.len

In [82]:
# Сгенерируем обучающий набор данных
traindata = Data(X_train, y_train.to_numpy())

In [83]:
batch_size = 1024
# Загрузите тренировочные данные в загрузчик данных с помощью respective batch_size and num_workers
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True, num_workers=0)

In [89]:
class LinearRegression(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(10, 100),
      nn.ReLU(),
      nn.Linear(100, 50),
      nn.ReLU(),
      nn.Linear(50, 25), 
      nn.ReLU(),
      nn.Linear(25, 1), 
      nn.Sigmoid()
    )
  
  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [90]:
model = LinearRegression()
model

LinearRegression(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [102]:
# критерий для вычисления потерь между input and target
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss() # Создает критерий, который измеряет the Binary Cross Entropy between the target and the input probabilities
# оптимизатор, который будет использоваться для обновления весов и смещений
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

criterion = nn.BCEWithLogitsLoss()
- This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability. 

In [103]:
model.train()

LinearRegression(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [104]:
# Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
        # Get and prepare inputs
     
        inputs, targets = data
        
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
      
        # Zero the gradients
        optimizer.zero_grad()
      
        # Perform forward pass
        outputs = model(inputs)
      
        # Compute loss
        loss = criterion(outputs, targets)
      
        # Perform backward pass
        loss.backward()
      
        # Perform optimization
        optimizer.step()
      
        # Print statistics
        current_loss += loss.item()
        if i % 10 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 0.013
Loss after mini-batch    11: 0.042
Loss after mini-batch    21: 0.006
Loss after mini-batch    31: 0.005
Loss after mini-batch    41: 0.004
Loss after mini-batch    51: 0.004
Loss after mini-batch    61: 0.004
Loss after mini-batch    71: 0.004
Loss after mini-batch    81: 0.004
Loss after mini-batch    91: 0.004
Loss after mini-batch   101: 0.004
Starting epoch 2
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.004
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.004
Loss after mini-batch    51: 0.004
Loss after mini-batch    61: 0.004
Loss after mini-batch    71: 0.004
Loss after mini-batch    81: 0.004
Loss after mini-batch    91: 0.004
Loss after mini-batch   101: 0.004
Starting epoch 3
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.004
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.004
Loss

In [105]:
testdata = Data(X_test, y_test.to_numpy())
testloader = DataLoader(testdata, batch_size=batch_size, 
                        shuffle=True, num_workers=0)

In [123]:
model.eval()
num_correct = 0
num_examples = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(testloader):
        predictions = model(inputs)
        labels = torch.from_numpy(labels.detach().cpu().numpy())
        predictions = torch.from_numpy(predictions.detach().cpu().numpy())
        correct = torch.eq(torch.max(F.sigmoid(predictions), dim=1)[1], labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
print("Точность результатов Accuracy %.2f%%" % ( 100 * num_correct / num_examples))

Точность результатов Accuracy 93.32%


# Сравнение Torch и Tensorflow.keras 



Точность результатов +/- совпадает, отличия незначительные.
Torch быстрее работает, за счет этого выигрывает. В основном всё "из коробки"

#### Хороший пример Обучение модели классификатора изображений с помощью PyTorch
https://learn.microsoft.com/ru-ru/windows/ai/windows-ml/tutorials/pytorch-train-model

## Модель 1

## Создаем нейронную сеть

* Первая строка создает Sequential модель. Это самый простой вид модели Keras для нейронных сетей, которые просто состоят из одного стека слоев, соединенных последовательно. Это называется последовательным API.
* Далее мы строим первый слой и добавляем его в модель. Это слой Flatten - этот слой не имеет никаких параметров; это просто сделать некоторую простую предварительную обработку. Поскольку это первый слой в модели, вы должны указать input_shape , который не включает размер пакета, а только форму экземпляров.
* Затем мы добавляем Dense скрытый слой с 30 нейронами. Он будет использовать функцию активации ReLU. relu(...): Применяет функцию активации выпрямленного линейного блока. Каждый Dense слой управляет своей собственной весовой матрицей, содержащей все веса соединений между нейронами и их входами. Он также управляет вектором терминов смещения (по одному на нейрон). Когда он получает некоторые входные данные, он вычисляет уравнение 10-2 .
* Затем мы добавляем второй плотный скрытый слой со 10 нейронами, также используя функцию активации ReLU.

* Наконец, мы добавляем Dense выходной слой с 1 нейроном, используя сигмовидную функцию активации sigmoid(...): сигмовидная функция активации, sigmoid(x) = 1 / (1 + exp(-x)).
 https://www.tensorflow.org/api_docs/python/tf/keras/activations

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = X_train.shape[1:]),#входной слой. Flatten Выравнивает ввод. Не влияет на размер партии.
    tf.keras.layers.Dense(30, activation="relu"), # скрытый слой
    tf.keras.layers.Dense(10, activation="relu"), # скрытый плотный слой
    tf.keras.layers.Dense(1, activation="sigmoid") #выходной слой
])


In [30]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 10)                0         
                                                                 
 dense_15 (Dense)            (None, 30)                330       
                                                                 
 dense_16 (Dense)            (None, 10)                310       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


* optimizer https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
В этом алгоритме оптимизации используются скользящие средние как градиентов, так и вторых моментов градиентов. 
* class BinaryCrossentropy: вычисляет кросс-энтропийную потерю между истинными метками и предсказанными метками.
 https://www.tensorflow.org/api_docs/python/tf/keras/losses

## Обучаем нейронную сеть

* batch_size - Целое или None. Количество выборок на пакет вычислений. Если не указано, по умолчанию будет 32. Не указывайте, если ваши данные представлены в виде набора данных, генераторов или экземпляров (поскольку они генерируют пакеты). batch_sizebatch_sizekeras.utils.Sequence

* epochs - Целое. Количество эпох для обучения модели. Эпоха — это итерация по всем x и y предоставленным данным (если только the steps_per_epoch flag флаг не установлен в значение, отличное от None). Обратите внимание, что в сочетании с initial_epoch, epochs следует понимать как «последняя эпоха». Модель не обучается для количества итераций, заданного , а только до тех пор, пока не будет достигнута эпоха индекса .steps_per_epochinitial_epochepochsepochsepochs


In [32]:
%%time
history = model.fit(X_train_scaled, y_train, batch_size=None, epochs=20, validation_data=(X_test_scaled, y_test))

Epoch 1/20
3282/3282 [==============================] - 20s 6ms/step - loss: 0.2134 - accuracy: 0.9308 - val_loss: 0.1884 - val_accuracy: 0.9353
Epoch 2/20
3282/3282 [==============================] - 18s 6ms/step - loss: 0.1914 - accuracy: 0.9353 - val_loss: 0.1864 - val_accuracy: 0.9354
Epoch 3/20
3282/3282 [==============================] - 18s 6ms/step - loss: 0.1904 - accuracy: 0.9354 - val_loss: 0.1862 - val_accuracy: 0.9357
Epoch 4/20
3282/3282 [==============================] - 18s 6ms/step - loss: 0.1898 - accuracy: 0.9355 - val_loss: 0.1855 - val_accuracy: 0.9358
Epoch 5/20
3282/3282 [==============================] - 19s 6ms/step - loss: 0.1895 - accuracy: 0.9359 - val_loss: 0.1863 - val_accuracy: 0.9365
Epoch 6/20
3282/3282 [==============================] - 19s 6ms/step - loss: 0.1889 - accuracy: 0.9362 - val_loss: 0.1857 - val_accuracy: 0.9368
Epoch 7/20
3282/3282 [==============================] - 19s 6ms/step - loss: 0.1886 - accuracy: 0.9363 - val_loss: 0.1867 - val_ac

In [33]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC работы на тестовых данных: %.2f%%" % (scores[1]*100))

AUC работы на тестовых данных: 93.46%


In [34]:
%%time
history = model.fit(X_train_scaled, y_train, batch_size=128, epochs=20, validation_data=(X_test_scaled, y_test))

Epoch 1/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1859 - accuracy: 0.9371 - val_loss: 0.1848 - val_accuracy: 0.9365
Epoch 2/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1858 - accuracy: 0.9371 - val_loss: 0.1847 - val_accuracy: 0.9366
Epoch 3/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1858 - accuracy: 0.9372 - val_loss: 0.1850 - val_accuracy: 0.9366
Epoch 4/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1858 - accuracy: 0.9370 - val_loss: 0.1845 - val_accuracy: 0.9367
Epoch 5/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1858 - accuracy: 0.9370 - val_loss: 0.1866 - val_accuracy: 0.9361
Epoch 6/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1858 - accuracy: 0.9370 - val_loss: 0.1852 - val_accuracy: 0.9364
Epoch 7/20
821/821 [==============================] - 5s 6ms/step - loss: 0.1857 - accuracy: 0.9371 - val_loss: 0.1858 - val_accuracy: 0.9362
Epoch 

In [35]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC работы на тестовых данных: %.2f%%" % (scores[1]*100))

AUC работы на тестовых данных: 92.46%


In [36]:
%%time
history = model.fit(X_train, y_train, batch_size=1024, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
103/103 [==============================] - 1s 6ms/step - loss: 30.2253 - accuracy: 0.8940 - val_loss: 14.6940 - val_accuracy: 0.9233
Epoch 2/20
103/103 [==============================] - 0s 5ms/step - loss: 13.0632 - accuracy: 0.8861 - val_loss: 22.5982 - val_accuracy: 0.9344
Epoch 3/20
103/103 [==============================] - 0s 5ms/step - loss: 10.6981 - accuracy: 0.8932 - val_loss: 8.0996 - val_accuracy: 0.3177
Epoch 4/20
103/103 [==============================] - 1s 5ms/step - loss: 10.3098 - accuracy: 0.8847 - val_loss: 2.9607 - val_accuracy: 0.9322
Epoch 5/20
103/103 [==============================] - 1s 5ms/step - loss: 7.6398 - accuracy: 0.8855 - val_loss: 11.7731 - val_accuracy: 0.9340
Epoch 6/20
103/103 [==============================] - 1s 5ms/step - loss: 6.6897 - accuracy: 0.8858 - val_loss: 4.0099 - val_accuracy: 0.9328
Epoch 7/20
103/103 [==============================] - 1s 6ms/step - loss: 8.7499 - accuracy: 0.8840 - val_loss: 10.5793 - val_accuracy: 0.934

In [37]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC работы на тестовых данных: %.2f%%" % (scores[1]*100))

AUC работы на тестовых данных: 93.44%


## Модель 2

In [38]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = X_train.shape[1:]),#входной слой. Flatten Выравнивает ввод. Не влияет на размер партии.
    tf.keras.layers.Dense(30, activation="relu"), # скрытый слой
    tf.keras.layers.Dense(1, activation="sigmoid") #выходной слой
])


In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 10)                0         
                                                                 
 dense_18 (Dense)            (None, 30)                330       
                                                                 
 dense_19 (Dense)            (None, 1)                 31        
                                                                 
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
%%time
history = model.fit(X_train, y_train, batch_size=1024, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
103/103 [==============================] - 1s 8ms/step - loss: 86.7736 - accuracy: 0.7554 - val_loss: 5.0075 - val_accuracy: 0.2693
Epoch 2/20
103/103 [==============================] - 1s 7ms/step - loss: 1.5499 - accuracy: 0.8894 - val_loss: 0.7881 - val_accuracy: 0.9050
Epoch 3/20
103/103 [==============================] - 1s 6ms/step - loss: 0.7460 - accuracy: 0.9137 - val_loss: 0.6087 - val_accuracy: 0.8982
Epoch 4/20
103/103 [==============================] - 1s 6ms/step - loss: 0.6528 - accuracy: 0.9145 - val_loss: 0.4807 - val_accuracy: 0.9336
Epoch 5/20
103/103 [==============================] - 1s 6ms/step - loss: 0.8982 - accuracy: 0.9050 - val_loss: 0.6449 - val_accuracy: 0.9331
Epoch 6/20
103/103 [==============================] - 1s 6ms/step - loss: 0.5003 - accuracy: 0.9222 - val_loss: 1.0484 - val_accuracy: 0.9345
Epoch 7/20
103/103 [==============================] - 1s 6ms/step - loss: 1.2305 - accuracy: 0.8964 - val_loss: 1.0729 - val_accuracy: 0.9340
Epoch

In [44]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC работы на тестовых данных: %.2f%%" % (scores[1]*100))

AUC работы на тестовых данных: 30.78%


## Модель 3

In [45]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = X_train.shape[1:]),#входной слой. Flatten Выравнивает ввод. Не влияет на размер партии.
    tf.keras.layers.Dense(30, activation="relu"), # скрытый слой
    tf.keras.layers.Dense(20, activation="relu"), # плотный слой
    tf.keras.layers.Dense(10, activation="relu"), # плотный слой
    tf.keras.layers.Dense(1, activation="sigmoid") #выходной слой
])


In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 10)                0         
                                                                 
 dense_20 (Dense)            (None, 30)                330       
                                                                 
 dense_21 (Dense)            (None, 20)                620       
                                                                 
 dense_22 (Dense)            (None, 10)                210       
                                                                 
 dense_23 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,171
Trainable params: 1,171
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
%%time
history = model.fit(X_train, y_train, batch_size=1024, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
103/103 [==============================] - 1s 8ms/step - loss: 30.1739 - accuracy: 0.8322 - val_loss: 1.1339 - val_accuracy: 0.9247
Epoch 2/20
103/103 [==============================] - 1s 6ms/step - loss: 1.0109 - accuracy: 0.8816 - val_loss: 1.2032 - val_accuracy: 0.9275
Epoch 3/20
103/103 [==============================] - 1s 5ms/step - loss: 0.8460 - accuracy: 0.8845 - val_loss: 1.6905 - val_accuracy: 0.9333
Epoch 4/20
103/103 [==============================] - 1s 5ms/step - loss: 1.6445 - accuracy: 0.8811 - val_loss: 0.4900 - val_accuracy: 0.9332
Epoch 5/20
103/103 [==============================] - 1s 5ms/step - loss: 1.7332 - accuracy: 0.8916 - val_loss: 0.4877 - val_accuracy: 0.9343
Epoch 6/20
103/103 [==============================] - 1s 5ms/step - loss: 1.9869 - accuracy: 0.8872 - val_loss: 2.0864 - val_accuracy: 0.9324
Epoch 7/20
103/103 [==============================] - 1s 5ms/step - loss: 1.6000 - accuracy: 0.8730 - val_loss: 2.4958 - val_accuracy: 0.9348
Epoch

In [49]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC работы на тестовых данных: %.2f%%" % (scores[1]*100))

AUC работы на тестовых данных: 93.56%


# Вывод
* от размера выборки batch_size  и количества эпох epochs +- результаты не меняются, отличаются незначительно. Лучший резльтат AUS 93.56%. Существенно отличается только время, при большом количестве эпох или малом размере выборки время увеличивается.
* Если не делать плотный слой, то очень падает метрика AUC работы на тестовых данных: 30.78%.
* При увеличении плотных слоев ничего не меняетсяя.